In [3]:
!pip install selenium
!pip install bs4
!pip install requests
!pip install urllib3
!pip install webdriver_manager
!pip install openpyxl
!pip install xlrd
!pip install datetime
!pip install lxml
!pip install pandas
#다 한 다음에 주피터 노트북 완전 껐다가 재 실행(이 브라우저 창만 끄지말고 아예 꺼야함)

In [ ]:
## 필요한 패키지 로딩 
import time
start_time = time.time() 
import datetime
import requests
import selenium
import lxml
import re
import warnings 
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver

: 

In [4]:
import pandas as pd

# 엑셀 파일 읽기
file_path = r"/Users/changjin/Desktop/Workspace/lab/sac/cafe/naver_cafe_list.xlsx"
df = pd.read_excel(file_path)
print(df.shape)
df.tail()

(226, 3)


,no,store_name,store_url_naver
221,222,버떠,https://m.place.naver.com/restaurant/150031860...
222,223,백금당 강릉점,https://m.place.naver.com/restaurant/150429132...
223,224,김성수제빵소 강릉점,https://m.place.naver.com/restaurant/175818105...
224,225,강릉옥수수빵,https://m.place.naver.com/restaurant/138161389...
225,226,테라로사 강릉 신라모노그램점,https://m.place.naver.com/restaurant/206425900...


---
##### max_scrolls : 페이지 펼침 횟수 지정, 한 페이지에 10개의 리뷰가 등록되므로 max_scrolls = 10이라면 최근 100개의 리뷰를 가져오게 된다. 
---

In [5]:
################################### 페이지 펼침 수 지정 ################
max_scrolls = 1
###########################################################################################

In [6]:
import re, json, time, datetime
from pathlib import Path

import pandas as pd
import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# ---------------------------
# 날짜를 YYYYMMDD int로
# ---------------------------
def parse_date_to_yyyymmdd(s: str, now=None):
    if not s:
        return None
    s = str(s).strip()
    if now is None:
        now = datetime.datetime.now()

    m = re.search(r'(\d{4})[.\-/](\d{1,2})[.\-/](\d{1,2})', s)
    if m:
        y, mth, d = int(m.group(1)), int(m.group(2)), int(m.group(3))
        try:
            return int(datetime.date(y, mth, d).strftime('%Y%m%d'))
        except:
            pass

    if '오늘' in s:
        return int(now.strftime('%Y%m%d'))
    if '어제' in s:
        return int((now - datetime.timedelta(days=1)).strftime('%Y%m%d'))
    m = re.search(r'(\d+)\s*일\s*전', s)
    if m:
        days = int(m.group(1))
        return int((now - datetime.timedelta(days=days)).strftime('%Y%m%d'))

    # "7.18." 같이 연도 없는 형식 → 올해로 보정
    m = re.search(r'(\d{1,2})[.]\s*(\d{1,2})[.]', s)
    if m:
        y = now.year
        mth, d = int(m.group(1)), int(m.group(2))
        try:
            return int(datetime.date(y, mth, d).strftime('%Y%m%d'))
        except:
            pass

    # 숫자 8자리만 온 경우
    if re.fullmatch(r'\d{8}', s):
        return int(s)
    return None


# ---------------------------
# place_id: URL에서 숫자
# ---------------------------
def extract_place_id(url: str):
    m = re.search(r'/restaurant/(\d+)', url)
    return m.group(1) if m else None


# ---------------------------
# Home: 가게명, 카테고리, 방문자/블로그 리뷰수, 설명(XtBbS)
# ---------------------------
def scrape_home(driver, home_url):
    driver.get(home_url)
    driver.implicitly_wait(5)

    place_name = None
    category = None
    visitor_count, blog_count = None, None
    soup = None

    try:
        WebDriverWait(driver, 6).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#_title'))
        )
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # #_title 하위 span들: [0]=상호, [1]=카테고리 (스크린샷 기준)
        spans = soup.select('#_title span')
        if spans:
            place_name = spans[0].get_text(strip=True) if len(spans) >= 1 else None
            category   = spans[1].get_text(strip=True) if len(spans) >= 2 else None

        # 보강: 클래스 기반 (상호=GHAhO, 카테고리=lnJfT)
        if not place_name:
            el = soup.select_one('#_title span.GHAhO')
            if el: place_name = el.get_text(strip=True)
        if not category:
            el = soup.select_one('#_title span.lnJfT')
            if el: category = el.get_text(strip=True)
    except:
        pass

    # 방문자/블로그 리뷰수
    try:
        if soup is None:
            soup = BeautifulSoup(driver.page_source, 'lxml')
        a_visitor = soup.select_one('a[href*="/review/visitor"]')
        if a_visitor:
            t = a_visitor.get_text(" ", strip=True)
            m = re.search(r'(\d[\d,]*)', t)
            if m: visitor_count = int(m.group(1).replace(',', ''))

        a_blog = soup.select_one('a[href*="/review/ugc"]')
        if a_blog:
            t = a_blog.get_text(" ", strip=True)
            m = re.search(r'(\d[\d,]*)', t)
            if m: blog_count = int(m.group(1).replace(',', ''))
    except:
        pass

    # 부가 설명(XtBbS)
    description = None
    try:
        if soup is None:
            soup = BeautifulSoup(driver.page_source, 'lxml')
        desc_div = soup.select_one('div.XtBbS')
        if desc_div:
            description = desc_div.get_text(" ", strip=True)
    except:
        pass

    return place_name, category, visitor_count, blog_count, description


# ---------------------------
# 리뷰(방문자): 추천순 클릭 후 상위 N개 (본문만 추출)
# ---------------------------
CONTENT_SELECTORS = [
    # 펼쳐진 본문(“접기”) 버튼 내부: 보통 여기에 전체 텍스트가 노출됨
    'a[role="button"][data-pui-click-code*="howless"]',
    # 아직 안 펼쳤을 때 버튼 내부 텍스트(간혹 본문 전체가 들어오기도 함)
    'a[role="button"][data-pui-click-code*="rvsho"]',
    # 과거 구조(예시)
    'div.pui__vn15t2 a[role="button"]',
    # 일반 div/p 폴백
    'div',
    'p',
]

def extract_review_text_from_li(li):
    for sel in CONTENT_SELECTORS:
        node = li.select_one(sel)
        if node and node.get_text(strip=True):
            txt = node.get_text(" ", strip=True)
            # 프로필/메타(“리뷰 123 사진 45 …”)가 들어온 경우를 걸러내기 위한 간단 휴리스틱:
            # 문장부호·조사·어절이 거의 없으면 스킵
            if re.search(r'[.!?…]|[가-힣]{2,}', txt):
                return re.sub(r'\s+', ' ', txt)
    return None

def scrape_reviews_recommended(driver, visitor_url, n=10, max_scrolls=6):
    driver.get(visitor_url)
    driver.implicitly_wait(5)

    # 추천순 클릭
    try:
        btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//a[@role="button"][contains(., "추천순")]'))
        )
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(1.0)
    except:
        pass

    # 스크롤하면서 각 li 안의 더보기(rvsho*) 모두 클릭
    for _ in range(max_scrolls):
        try:
            more_btns = driver.find_elements(
                By.XPATH,
                '//a[@role="button" and contains(@data-pui-click-code,"rvsho")]'
            )
            for b in more_btns:
                try:
                    driver.execute_script("arguments[0].click();", b)
                    time.sleep(0.05)
                except:
                    pass
        except:
            pass
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.6)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    items = soup.select('ul#_review_list > li')
    reviews = []

    for li in items:
        # 본문만 가져오기
        text = extract_review_text_from_li(li)

        # 시간
        t_el = li.select_one('time[aria-hidden="true"]') or li.find('time')
        t_txt = t_el.get_text(strip=True) if t_el else None
        t_int = parse_date_to_yyyymmdd(t_txt)

        if text:
            reviews.append({"time": t_int, "text": text})
        if len(reviews) >= n:
            break

    return reviews


# ---------------------------
# 메뉴 탭: 메뉴 리스트
# ---------------------------
def scrape_menu(driver, menu_url, max_scrolls=4):
    driver.get(menu_url)
    driver.implicitly_wait(5)

    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.6)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    menu_items = []

    # 스크린샷 기준: div.place_section_content 내부 li.E2jtL
    lis = soup.select('div.place_section_content li.E2jtL')
    for li in lis:
        name = None
        for c in [li.select_one('a[role="button"]'), li.select_one('div'), li]:
            if c:
                txt = c.get_text(" ", strip=True)
                if txt:
                    name = re.sub(r'\s+', ' ', txt)
                    break
        if name:
            menu_items.append(name)

    # 중복 제거
    seen, dedup = set(), []
    for item in menu_items:
        if item not in seen:
            seen.add(item)
            dedup.append(item)

    return dedup


# ===========================================================
# 코랩 루프: df['store_name'], df['store_url_naver'] 사용
# ===========================================================

# df = pd.read_csv('stores.csv')  # 예시
out_file = Path('cafe_all_places.jsonl')
max_scrolls_reviews = 6

for x in range(len(df['store_name'])):
    url = str(df['store_url_naver'][x]).strip()
    s_store = str(df['store_name'][x]).strip()
    place_id = extract_place_id(url)

    # WebDriver (원래 세팅과 동일)
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_argument('--force-device-scale-factor=0.4')
    options.add_argument("disable-gpu")

    # 보조 requests 세션(선택)
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(5)

        # ---------- Home ----------
        category = None
        visiter_review_count = None
        blog_review_count = None
        description = None
        place_name = s_store

        if place_id:
            home_url = f"https://m.place.naver.com/restaurant/{place_id}/home"
            p_name, category, v_cnt, b_cnt, description = scrape_home(driver, home_url)
            if p_name: place_name = p_name
            if v_cnt is not None: visiter_review_count = int(v_cnt)
            if b_cnt is not None: blog_review_count = int(b_cnt)

        # ---------- Reviews: 추천순 상위 10 (본문만) ----------
        reviews_attraction = []
        if place_id:
            visitor_url = f"https://m.place.naver.com/restaurant/{place_id}/review/visitor?entry=ple"
            reviews_attraction = scrape_reviews_recommended(
                driver, visitor_url, n=10, max_scrolls=max_scrolls_reviews
            )

        # ---------- Menu ----------
        menu_list = []
        if place_id:
            menu_url = f"https://m.place.naver.com/restaurant/{place_id}/menu"
            menu_list = scrape_menu(driver, menu_url, max_scrolls=4)

        # ---------- JSONL 저장 ----------
        record = {
            "place_id": place_id,
            "place_name": place_name,
            "category": category,      # home에서 추출
            "min_price": None,
            "max_price": None,
            "avg_price": None,
            "visiter_review_count": visiter_review_count,
            "blog_review_count": blog_review_count,
            "description": description,
            "menu": menu_list,
            "reviews_attraction": reviews_attraction
        }

        with open(out_file, "a", encoding="utf-8") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

        print(f"[OK] {x} {place_name} ({place_id})  "
              f"Cat:{category}  V:{visiter_review_count}  B:{blog_review_count}  "
              f"reviews:{len(reviews_attraction)}  menu:{len(menu_list)}")

    except Exception as e:
        print(f"[ERROR] {s_store}: {e}")

    finally:
        try:
            driver.quit()
        except:
            pass


[OK] 0 카페제니엘 주문진점 (1543529590)  Cat:카페,디저트  V:5182  B:203  reviews:10  menu:15
[OK] 1 어게인디엠카페 (1195126769)  Cat:카페,디저트  V:1134  B:82  reviews:10  menu:6
[OK] 2 카페초이 (1606960077)  Cat:카페,디저트  V:861  B:178  reviews:10  menu:10
[OK] 3 에이엠브레드앤커피 (34203501)  Cat:베이커리  V:5855  B:2870  reviews:10  menu:15
[OK] 4 강릉빵다방 (66343890)  Cat:베이커리  V:4433  B:2225  reviews:10  menu:5
[OK] 5 강릉샌드 본점 (2051244903)  Cat:카페,디저트  V:246  B:186  reviews:10  menu:8
[OK] 6 카페제니엘 주문진점 (1543529590)  Cat:카페,디저트  V:5182  B:203  reviews:10  menu:15
[OK] 7 정남미명과 강릉본점 (1220981140)  Cat:카페,디저트  V:2535  B:392  reviews:10  menu:6
[OK] 8 글루크 (2031533892)  Cat:카페,디저트  V:401  B:31  reviews:10  menu:11
[OK] 9 카페 툇마루 (998885728)  Cat:카페,디저트  V:7704  B:8491  reviews:10  menu:15
[OK] 10 미르마르 (1933690279)  Cat:카페,디저트  V:2557  B:1321  reviews:10  menu:15
[OK] 11 초당밀 (1188561598)  Cat:카페  V:1748  B:623  reviews:10  menu:15
[OK] 12 플레이스 플러스 (15294021)  Cat:플레이스 플러스  V:5373  B:1261  reviews:10  menu:15
[OK] 13 그다이마잇 (1033478410)  Cat